In [9]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary

In [2]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Download and load the training data
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten the image
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

In [12]:
net

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)

In [10]:
summary(net, input_size=(64, 1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
Net                                      [64, 10]                  --
├─Linear: 1-1                            [64, 512]                 401,920
├─Linear: 1-2                            [64, 256]                 131,328
├─Linear: 1-3                            [64, 10]                  2,570
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
Total mult-adds (M): 34.29
Input size (MB): 0.20
Forward/backward pass size (MB): 0.40
Params size (MB): 2.14
Estimated Total Size (MB): 2.74

In [5]:
for epoch in range(5):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

[Epoch 1, Batch 100] loss: 0.760
[Epoch 1, Batch 200] loss: 0.404
[Epoch 1, Batch 300] loss: 0.295
[Epoch 1, Batch 400] loss: 0.276
[Epoch 1, Batch 500] loss: 0.254
[Epoch 1, Batch 600] loss: 0.236
[Epoch 1, Batch 700] loss: 0.195
[Epoch 1, Batch 800] loss: 0.192
[Epoch 1, Batch 900] loss: 0.177
[Epoch 2, Batch 100] loss: 0.136
[Epoch 2, Batch 200] loss: 0.135
[Epoch 2, Batch 300] loss: 0.141
[Epoch 2, Batch 400] loss: 0.145
[Epoch 2, Batch 500] loss: 0.140
[Epoch 2, Batch 600] loss: 0.140
[Epoch 2, Batch 700] loss: 0.137
[Epoch 2, Batch 800] loss: 0.134
[Epoch 2, Batch 900] loss: 0.126
[Epoch 3, Batch 100] loss: 0.096
[Epoch 3, Batch 200] loss: 0.116
[Epoch 3, Batch 300] loss: 0.100
[Epoch 3, Batch 400] loss: 0.100
[Epoch 3, Batch 500] loss: 0.101
[Epoch 3, Batch 600] loss: 0.097
[Epoch 3, Batch 700] loss: 0.092
[Epoch 3, Batch 800] loss: 0.097
[Epoch 3, Batch 900] loss: 0.106
[Epoch 4, Batch 100] loss: 0.075
[Epoch 4, Batch 200] loss: 0.074
[Epoch 4, Batch 300] loss: 0.098
[Epoch 4, 

In [11]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

Accuracy of the network on the 10000 test images: 97.04%
